# [DevNet Associate Model-Driven Programmability - NETCONF](https://learningnetwork.cisco.com/s/devnet-associate-exam-topics)
### 3.8 Apply concepts of model driven programmability (YANG, RESTCONF, and NETCONF) in a Cisco environment
### 5.1 Describe the value of model driven programmability for infrastructure automation
### 5.10 Interpret the results of a RESTCONF or NETCONF query
### 5.11 Interpret basic YANG models

---

### Tasks:
1. Review Lab Environment.
2. Explore YANG Models Using Advanced NETCONF Explorer (ANX).
3. Interact with IOS-XE YANG Modules Using Python **ncclient**.

---

### Review Lab Environment
The Programmability Foundations Lab has two CSR1000v devices which run IOS-XE 16.2(2a). Access information for these devices is as follows:
1. **Router 1 (R1)** - r1.lab.local (192.168.2.161)
2. **Router 2 (R2)** - r2.lab.local (192.168.2.162)
3. **Username** - wwt
4. **Password** - WWTwwt1!
 
 
 The [Programmability Foundations Lab Guide](https://labs.wwtatc.com/lab-guides/programmability_foundations_lab/index.html) has complete lab topology information.

#### Connect to R1 & R2
1. Click the **MTPuTTY** icon in the taskbar.
2. Expand the **PuTTY Sessions** folder.
3. Double-click on each **r1.lab.local** and **r2.lab.local** to establish SSH sessions.
4. When prompted, log on to both routers.

![1_mtputty.png](_images/1_mtputty.png)

#### Disable Session Timeout on R2
SSH sessions to **R1** have no time limit although SSH sessions to **R2** expire after 10 minutes of inactivity. Use the following CLI commands to disable session timeout:

```
configure terminal
!
line vty 0 15
 exec-timeout 0
 end
wr mem
```

#### Review R1 NETCONF Configuration
Use the following CLI commands to review information about the NETCONF configuration on R1:

```
show run | include netconf
```

* The result of this command shows the single IOS-XE command **netconf-yang** which enables NETCONF on the device.
* The command **no netconf-yang**, in global configuration (configure terminal) mode will disable NETCONF, platform-wide.

```
show netconf-yang datastores
show netconf-yang status
```

The results of these commands show that:
* The **running** datastore is available.
* NETCONF is enabled on the default port, 830.
* The **candidate** datastore is disabled.

```
show netconf schema
```

This command displays the XML schema for the device and allows you to view a hierarchical representation of the available RPCs and paramaters.

#### Review R2 NETCONF Configuration
Use the following CLI commands to review information about the NETCONF configuration on R2:

```
show run | include netconf
```

* The result of this command shows a line of configuration that is not in the R1 config, **netconf-yang feature candidate-datastore**.
* As the syntax implies, this command enables the **candidate** datastore.
* Use the following commands to validate the status of the **candidate** datastore:

```
show netconf-yang datastores
show netconf-yang status
```

The results of these commands show that:
* The **running** and **candidate** datastores are available.
* NETCONF is enabled on the default port, 830.
* The **candidate** datastore is enabled.
---

### Explore YANG Models Using Advanced NETCONF Explorer (ANX)
ANX allows you to interactively:
* Explore the YANG models of compatible devices.
 * ANX requires devices to support the NETCONF monitoring standard ([RFC6022](https://tools.ietf.org/html/rfc6022)).
* Create NETCONF payloads and filters.
* Send and receive NETCONF RPCs.

#### Connect to R1 with ANX
* ANX requires an online connecction to a network device.
* The lab setup process should have opened a second Chrome browser tab for ANX.
* If not, use the **ANX** desktop shortcut to open a new tab to ANX or manually browse to [http://localhost:9269](http://localhost:9269).

To log on to ANX:
1. Set the **NETCONF Host** field to **r1.lab.local**.
2. Set the **Username** to **wwt**.
3. Set the **Password** to **WWTwwt1!**.
4. Tick the boxes both to **Cache** YANG models and **Remember credentials**.
5. Click the **Login** button.

![2_anx_login.png](_images/2_anx_login.png)

#### Wait for ANX to Download YANG Models
* ANX will load all of the YANG models from R1 which will take 5-7 minutes.
* The ANX main view will load as soon as the YANG model download completes.

![3_anx_overview.png](_images/3_anx_overview.png)

#### Search for YANG Models
* Type **interfaces** in the **Search Models** field and press your **Enter** or **Return** key.
* Notice the search results display all available YANG models which contain the word **interface**, including:
 * Cisco IOS-XE **Native** Operational model.
 * **IETF** Configuration & Operational models.
 * **OpenConfig** models.
* To locate the Cisco IOS-XE **Native** Configuration model, search instead for **native**.

#### Explore YANG Models
* Click on the **ietf-interfaces** model and notice the ANX interface updates to show you details about this specific YANG element.
 * Since we selected a top-level module, ANX displays data for the first container in the hierarcchy.
* Notice details including:
 * The element **type**.
 * The **XPath**.
 * The **Subtree Filter** - **\*\* *this very useful data for Python interactions* \*\***
* These values will update as you navigate YANG models.

#### Explore Device Data
* ANX will show you configuration and state data for a device within the YANG model hierarchy.
* Click the **Show Data** button and choose the **Running** datastore from the menu.
* Expand the **ietf-interfaces** hierarchy including:
 * The **interface** list.
 * The **ipv4** container.
 * The **address** list.
* Notice the configuration data from R1 in the hierarchy.
* Click on the **address** element and notice the ANX display update to show, among other things, the **ietf-ip** namespace.

#### Explore the NETCONF Console
* ANX allows you to send and receive NETCONF payloads directly.
* Click the **ipv4** container in the hierarchyand then click the **NETCONF Console** button.
* In the NETCONF Console view, click the **\<get\>** button and then click the **Send Request** button.
* Notice how the NETCONF Console:
 * Creates a NETCONF XML payload.
 * Sends the payload to the device in an **\<rpc\>** message.
 * Displays the body of the **\<rpc-reply\>** message.
* The NETCONF Console allows you to manually edit XML payloads and choose between **\<get\>**, **\<edit-config\> merge**, **\<edit-config\> delete**, and **\<commit\>** operations.
 * **\*\* *This is another very useful way to build XML payloads for Python interactions* \*\***
---

### Interact with IOS-XE YANG Modules Using Python ncclient.
**ncclient** is a Python NETCCONF client module available on [PyPI](https://pypi.org/project/ncclient/) and pre-installed in this environment.  The core operations for NETCONF interactions are part of the [manager](https://ncclient.readthedocs.io/en/latest/manager.html) class in the **ncclient** module.

The following tasks walk you through:
* Creating NETCONF sessions to R1 & R2.
* Using ANX to build RPC payloads.
* Sending **\<rpc\>** messages to R1 & R2.
* Displaying data from **\<rpc\>** reply messages.

**The tasks rely heavily on the use of the **xmltodict** Python module, also pre-installed in this environment and available on [PyPi](https://pypi.org/project/xmltodict/).**

---
#### Import Modules

In [ ]:
from ncclient import manager
from pprint import pprint
import xmltodict

---
#### Create Dictionaries for Device Connection Properties

In [ ]:
r1 = {
    'host': 'r1.lab.local',
    'username': 'wwt',
    'password': 'WWTwwt1!',
    'hostkey_verify': False,
    'device_params': {
        'name':'csr'
    }
}
pprint(r1)

In [ ]:
r2 = {
    'host': 'r2.lab.local',
    'username': 'wwt',
    'password': 'WWTwwt1!',
    'hostkey_verify': False,
    'device_params': {
        'name':'csr'
    }
}
pprint(r2)

---
#### Retreive NETCONF Agent Capabilities from R1

In [ ]:
# Use a Context Manager with the "manager" class to handle session cleanup.
with manager.connect(**r1) as conn:
    caps = conn.server_capabilities

print(f'** {len(caps)} Total NETCONF Capabilities **')

---
#### Display the First 25 Capabilities

In [ ]:
for i in range(25):
    print(f'{i + 1}. {list(caps)[i]}')

---
#### Get Interface Operational Data
* Use ANX to build a filter as follows:
 * YANG Model - **Cisco-IOS-XE-interfaces-oper**
 * XPath - **/interfaces-ios-xe-oper:interfaces/interface/v4-protocol-stats**
 * Optional - filter on interface **GigabitEthernet1**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
  <interface>
    <v4-protocol-stats/>
    <name/>
  </interface>
 </interfaces>
</filter>
'''

In [ ]:
# Create a <get> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "get" method and set the "filter" keyword argument to add the XML payload to the RPC
    response = conn.get(filter=payload)

In [ ]:
# Display the type for the response object
print(type(response))

In [ ]:
# Display the properties of the response object
properties = [prop for prop in dir(response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the raw XML response, in the 'xml' property of the "response" object
print(response.xml)

In [ ]:
# Display the 'data_xml' property of the "response" object
print(response.data_xml)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display state information in a more readable format
interface_data = py_response['data']['interfaces']['interface']
for inter in interface_data:
    print(f'Interface: {inter["name"]}')
    print(f'\tPackets in: {inter["v4-protocol-stats"]["in-pkts"]}')
    print(f'\tPackets out: {inter["v4-protocol-stats"]["out-pkts"]}\n')

---
#### Get Interface Configuration Data
* Use ANX to build a filter as follows:
 * YANG Model - **ietf-interfaces**
 * XPath - **/if:interfaces/interface/ip:ipv4**
 * Optional - filter on interface **GigabitEthernet1**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    <name/>
  </interface>
 </interfaces>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
    response = conn.get_config(source='running', filter=payload)

In [ ]:
# Display the type for the response object
print(type(response))

In [ ]:
# Display the 'data_xml' property of the "response" object
print(response.data_xml)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['interfaces']['interface']
for inter in interface_config:
    print(f'Interface: {inter["name"]}')
    print(f'\tIP Address: {inter["ipv4"]["address"]["ip"]}')
    print(f'\tSubnet Mask: {inter["ipv4"]["address"]["netmask"]}\n')

---
#### Create a New Interface
* Use ANX to build a configuration for a new Loopback interrface as follows:
 * YANG Model - **Cisco-IOS-XE-native**
 * XPath - **/ios:native/interface/Loopback/name**
 * Interface Name - **0**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<config>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <Loopback>
      <name>0</name>
    </Loopback>
  </interface>
 </native>
</config>
'''

In [ ]:
# Create an <edit-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "edit_config" method, set the "target" keyword argument to "running", and set the "config" keyword argument to add the XML payload to the RPC
    response = conn.edit_config(target='running', config=payload)

In [ ]:
# Display the properties of the response object
properties = [prop for prop in dir(response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Verify Configuration Change

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
 </native>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
    response = conn.get_config(source='running', filter=payload)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['native']['interface']
for key, value in interface_config.items():
    if type(value) == list:
        print(f'Interface: {key}{value[0]["name"]}')
        print(f'\tIP Address: {value[0]["ip"]["address"]["primary"]["address"]}\n')
    else:
        print(f'Interface: {key}{value["name"]}')
        if value.get('ip'):
            print(f'\tIP Address: {value["ip"]["address"]["primary"]["address"]}\n')
        else:
            print('\tIP Address: Not assigned')

---
#### Update Interface Configuration Details
* Use ANX to add an IP address, description, and load interval to the new Loopback interface:
 * YANG Model - **Cisco-IOS-XE-native**
 * XPath - **/ios:native/interface/Loopback/ip/address/primary**
 * IPv4 Address - **172.16.10.10**
 * Subnet Mask - **255.255.255.255**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<config>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <Loopback>
     <name>0</name>
     <description>Added by NETCONF</description>
     <load-interval>30</load-interval>
     <ip>
      <address>
        <primary>
          <address>172.16.10.10</address>
          <mask>255.255.255.255</mask>
        </primary>
      </address>
     </ip>
    </Loopback>
  </interface>
 </native>
</config>
'''

In [ ]:
# Create an <edit-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "edit_config" method, set the "target" keyword argument to "running", and set the "config" keyword argument to add the XML payload to the RPC
    response = conn.edit_config(target='running', config=payload)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Verify Configuration Change

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
 </native>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
    response = conn.get_config(source='running', filter=payload)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['native']['interface']
for key, value in interface_config.items():
    if type(value) == list:
        print(f'Interface: {key}{value[0]["name"]}')
        print(f'\tIP Address: {value[0]["ip"]["address"]["primary"]["address"]}\n')
    else:
        print(f'Interface: {key}{value["name"]}')
        print(f'\tDescription: {value.get("description", "None")}')
        if value.get('ip'):
            print(f'\tIP Address: {value["ip"]["address"]["primary"]["address"]}')
            print(f'\tSubnet Mask: {value["ip"]["address"]["primary"]["mask"]}\n')
        else:
            print('\tIP Address: Not assigned')

---
#### Delete the New Interface
* Use ANX to delete the new Loopback interface:
 * YANG Model - **Cisco-IOS-XE-native**
 * XPath - **/ios:native/interface/Loopback**
 * Interface Name - **0**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<config>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <Loopback operation="delete">
     <name>0</name>
    </Loopback>
  </interface>
 </native>
</config>
'''

In [ ]:
# Create an <edit-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "edit_config" method, set the "target" keyword argument to "running", and set the "config" keyword argument to add the XML payload to the RPC
    response = conn.edit_config(target='running', config=payload)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Verify Configuration Change

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
 </native>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
with manager.connect(**r1) as conn:
    # Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
    response = conn.get_config(source='running', filter=payload)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['native']['interface']
for key, value in interface_config.items():
    if type(value) == list:
        print(f'Interface: {key}{value[0]["name"]}')
        print(f'\tIP Address: {value[0]["ip"]["address"]["primary"]["address"]}\n')
    else:
        print(f'Interface: {key}{value["name"]}')
        print(f'\tDescription: {value.get("description", "None")}')
        if value.get('ip'):
            print(f'\tIP Address: {value["ip"]["address"]["primary"]["address"]}')
            print(f'\tSubnet Mask: {value["ip"]["address"]["primary"]["mask"]}\n')
        else:
            print('\tIP Address: Not assigned')

---
#### Edit the Candidate Datastore on R2
* IOS-XE blocks configuration changes to the **running** datastore when the **candidate** datastore is enabled. 
* Use ANX to build a configuration for a new Loopback interrface as follows:
 * YANG Model - **Cisco-IOS-XE-native**
 * XPath - **/ios:native/interface/Loopback/name**
 * Interface Name - **10**

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<config>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <Loopback>
      <name>10</name>
    </Loopback>
  </interface>
 </native>
</config>
'''

In [ ]:
# Try to create an <edit-config> RPC message with ncclient that targets the "running" datastore
from ncclient.operations.rpc import RPCError
with manager.connect(**r2) as conn:
    try:
        # Use the "edit_config" method, set the "target" keyword argument to "running", and set the "config" keyword argument to add the XML payload to the RPC
        response = conn.edit_config(target='running', config=payload)
    except RPCError as e:
        print(repr(e))
    

In [ ]:
# Try the same <edit-config> RPC message again but target the "candidate" datastore instead
# Don't use the context manager for this exercise, in order to keep the NETCONF session open for multiple operations
conn = manager.connect(**r2)
# Use the "edit_config" method, set the "target" keyword argument to "candidate", and set the "config" keyword argument to add the XML payload to the RPC
response = conn.edit_config(target='candidate', config=payload)  

In [ ]:
# Display the properties of the response object
properties = [prop for prop in dir(response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Verify Configuration Change
* Confirm the change to the **candidate** datastore does not appear in the **running** datastore, yet.

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
 </native>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
# Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
response = conn.get_config(source='running', filter=payload)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['native']['interface']
for key, value in interface_config.items():
    if type(value) == list:
        print(f'Interface: {key}{value[0]["name"]}')
        print(f'\tIP Address: {value[0]["ip"]["address"]["primary"]["address"]}\n')
    else:
        print(f'Interface: {key}{value["name"]}')
        if value.get('ip'):
            print(f'\tIP Address: {value["ip"]["address"]["primary"]["address"]}\n')
        else:
            print('\tIP Address: Not assigned')

---
#### Copy the Candidate Datastore on R2 to the Running Datastore

In [ ]:
# Create a <commit> RPC message with ncclient, to commit "candidate" datastore changes to the "running" datastore
# Use the "commit" method
response = conn.commit()

In [ ]:
# Display the properties of the response object
properties = [prop for prop in dir(response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Verify Configuration Change
* Commit the Commit to the Running Datastore on R2 is Successful.

In [ ]:
# Create a multi-line with an XML payload from ANX
payload = '''
<filter>
 <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
 </native>
</filter>
'''

In [ ]:
# Create a <get-config> RPC message with ncclient
# Use the "get_config" method, set the "source" keyword argument to "running", and set the "filter" keyword argument to add the XML payload to the RPC
response = conn.get_config(source='running', filter=payload)

In [ ]:
# Convert the 'response.data_xml' property to a Python dictionary
py_response = xmltodict.parse(
    response.data_xml,
    dict_constructor=dict
)

pprint(py_response)

In [ ]:
# Display configuration information in a more readable format
header_msg = '** Interface List **'
print(f'\n{"-" * len(header_msg)}')
print(header_msg)
print(f'{"-" * len(header_msg)}\n')
interface_config = py_response['data']['native']['interface']
for key, value in interface_config.items():
    if type(value) == list:
        print(f'Interface: {key}{value[0]["name"]}')
        print(f'\tIP Address: {value[0]["ip"]["address"]["primary"]["address"]}\n')
    else:
        print(f'Interface: {key}{value["name"]}')
        if value.get('ip'):
            print(f'\tIP Address: {value["ip"]["address"]["primary"]["address"]}\n')
        else:
            print('\tIP Address: Not assigned')

---
#### Close the NETCONF Session

In [ ]:
# Create a <close-session> RPC message with ncclient to gracefully close the NETCONF session to R2
response = conn.close_session()

In [ ]:
# Display the properties of the response object
properties = [prop for prop in dir(response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the 'xml' property of the "response" object
pprint(response.xml)

---
#### Save Running Configurations to Startup Configurations
* Write changes from the "running" datastore of both R1 & R2 to the "startup" datastore.

In [ ]:
# Create an XML string with the required payload to save the configuration
save_config_string = '<cisco-ia:save-config xmlns:cisco-ia="http://cisco.com/yang/cisco-ia"/>'

In [ ]:
# Import the "fromstring" method from the lxml.etree class, to convert the payload string into a properly-formatted XML element
from lxml.etree import fromstring
payload = fromstring(save_config_string)

In [ ]:
# Create <copy-config> RPC messages with ncclient, to copy the "running" datastore contents to the "startup" datastore, with both R1 & R2
with manager.connect(**r1) as conn:
    # Use the "dispatch" method and pass the XML-formatted payload as an argument
    r1_response = conn.dispatch(payload)

with manager.connect(**r2) as conn:
    # Use the "dispatch" method and pass the XML-formatted payload as an argument
    r2_response = conn.dispatch(payload)

In [ ]:
# Display the properties of one of the response objects
properties = [prop for prop in dir(r1_response) if prop[0] != '_' ]
print(properties)

In [ ]:
# Display the 'xml' property of the "r1_response" object
print(r1_response.xml)

In [ ]:
# Convert the 'r1_response.xml' and 'r2_response.xml' properties to Python dictionaries
py_r1_response = xmltodict.parse(
    r1_response.xml,
    dict_constructor=dict
)

py_r2_response = xmltodict.parse(
    r2_response.xml,
    dict_constructor=dict
)

pprint(py_r1_response)
pprint(py_r2_response)

In [ ]:
# Display the response information in a more readable format
print(f'Response from R1: {py_r1_response["rpc-reply"]["result"]["#text"]}')
print(f'Response from R2: {py_r2_response["rpc-reply"]["result"]["#text"]}')